In [ ]:
# Cell 1: Install Libraries
!pip install librosa soundfile numpy scikit-learn tensorflow matplotlib seaborn
print("All required libraries installed!")

In [ ]:
# Cell 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

In [ ]:
# Cell 3: Data Setup (Directly use unzipped folders)
import os
import shutil # Import shutil for robust directory management

print("--- Step 3: Data Setup (Using Pre-Unzipped Data) ---")

# --- VERY IMPORTANT: DEFINE YOUR UNZIPPED FOLDER PATHS CORRECTLY ---
# Example: If your unzipped 'Audio_Speech_Actors_01-24' and 'Audio_Song_Actors_01-24'
# folders are directly inside 'My Drive/emotion_audio/'
speech_data_dir = "/content/drive/MyDrive/emotion_audio/Audio_Speech_Actors_01-24"
song_data_dir = "/content/drive/MyDrive/emotion_audio/Audio_Song_Actors_01-24"

# Example: If your unzipped folders are in 'My Drive/My_Project_Data/Audio_Emotions/'
# speech_data_dir = "/content/drive/MyDrive/My_Project_Data/Audio_Emotions/Audio_Speech_Actors_01-24"
# song_data_dir = "/content/drive/MyDrive/My_Project_Data/Audio_Emotions/Audio_Song_Actors_01-24"

print(f"Set speech data directory to: {speech_data_dir}")
print(f"Set song data directory to: {song_data_dir}")

# --- Verification of directory existence ---
if not os.path.exists(speech_data_dir):
    print(f"ERROR: Speech data directory NOT FOUND at '{speech_data_dir}'. Please check the path carefully in Cell 3.")
    import sys
    sys.exit("Exiting due to speech data directory not found.")
else:
    print(f"SUCCESS: Speech data directory found: {speech_data_dir}")

if not os.path.exists(song_data_dir):
    print(f"ERROR: Song data directory NOT FOUND at '{song_data_dir}'. Please check the path carefully in Cell 3.")
    import sys
    sys.exit("Exiting due to song data directory not found.")
else:
    print(f"SUCCESS: Song data directory found: {song_data_dir}")

print("\n--- Verification: Listing contents of data directories ---")
print(f"Contents of {speech_data_dir}:")
list_speech_contents = os.listdir(speech_data_dir)
if list_speech_contents:
    print(f"  Found {len(list_speech_contents)} items (e.g., {list_speech_contents[0]}, ...)")
    # Check if a typical Actor folder exists and list its contents
    actor_dir_path = os.path.join(speech_data_dir, "Actor_01")
    if os.path.exists(actor_dir_path):
        print(f"  Contents of {actor_dir_path}:")
        actor_contents = os.listdir(actor_dir_path)
        if actor_contents:
            # Print first 5 or all if less than 5
            for i, item in enumerate(actor_contents[:5]):
                print(f"    - {item}")
            if len(actor_contents) > 5:
                print("    ...")
        else:
            print(f"    '{actor_dir_path}' is empty.")
    else:
        print(f"  'Actor_01' directory not found directly in '{speech_data_dir}'. Structure might be different.")
else:
    print(f"  '{speech_data_dir}' is empty.")

print(f"\nContents of {song_data_dir}:")
list_song_contents = os.listdir(song_data_dir)
if list_song_contents:
    print(f"  Found {len(list_song_contents)} items (e.g., {list_song_contents[0]}, ...)")
    actor_dir_path_song = os.path.join(song_data_dir, "Actor_01")
    if os.path.exists(actor_dir_path_song):
        print(f"  Contents of {actor_dir_path_song}:")
        actor_contents_song = os.listdir(actor_dir_path_song)
        if actor_contents_song:
            for i, item in enumerate(actor_contents_song[:5]):
                print(f"    - {item}")
            if len(actor_contents_song) > 5:
                print("    ...")
        else:
            print(f"    '{actor_dir_path_song}' is empty.")
    else:
        print(f"  'Actor_01' directory not found directly in '{song_data_dir}'. Structure might be different.")
else:
    print(f"  '{song_data_dir}' is empty.")
print("--- End Verification ---")
print("\n--- Step 3 Complete: Audio data paths set and verified. ---")

In [ ]:
# Cell 4: Data Loading and Metadata Extraction
import pandas as pd
import os

print("\n--- Step 4: Data Loading and Metadata Extraction ---")

# Function to extract emotion and other details from filename (RAVDESS dataset format)
def extract_file_metadata(file_path):
    # Example filename: 03-01-01-01-01-01-01.wav
    basename = os.path.basename(file_path)
    parts = basename.split('.')[0].split('-')

    emotion_codes = {
        '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
        '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
    }

    try:
        modality = int(parts[0])
        vocal_channel = int(parts[1])
        emotion_code = parts[2]
        emotion = emotion_codes.get(emotion_code, 'unknown')
        intensity = int(parts[3])
        statement = int(parts[4])
        repetition = int(parts[5])
        actor = int(parts[6])

        gender = 'male' if actor % 2 != 0 else 'female'

        return {
            'file_path': file_path,
            'emotion': emotion,
            'vocal_channel': 'speech' if vocal_channel == 1 else 'song',
            'intensity': intensity,
            'statement': statement,
            'actor': actor,
            'gender': gender
        }
    except (IndexError, ValueError) as e:
        print(f"WARNING: Could not parse filename '{basename}'. Skipping. Error: {e}")
        return None

audio_data = []

# Process speech files from the specified data directory
for root, _, files in os.walk(speech_data_dir):
    for file in files:
        if file.endswith('.wav'):
            file_path = os.path.join(root, file)
            metadata = extract_file_metadata(file_path)
            if metadata: # Only add if metadata extraction was successful
                audio_data.append(metadata)

# Process song files from the specified data directory
for root, _, files in os.walk(song_data_dir):
    for file in files:
        if file.endswith('.wav'):
            file_path = os.path.join(root, file)
            metadata = extract_file_metadata(file_path)
            if metadata:
                audio_data.append(metadata)

df = pd.DataFrame(audio_data)

# --- VERIFICATION STEP FOR DATAFRAME CONTENTS ---
print(f"Total audio files found and added to DataFrame: {len(df)}")
if df.empty:
    print("CRITICAL ERROR: DataFrame is empty. This means no WAV files were found or parsed correctly.")
    print("Please re-check Cell 3's output and verify the unzipped folder paths and contents.")
    import sys
    sys.exit("Exiting because DataFrame is empty.")
else:
    print("\nDataFrame head (first 5 rows):")
    print(df.head())
    print("\nDistribution of emotions in the dataset:")
    print(df['emotion'].value_counts())
    print("\n--- Step 4 Complete: DataFrame populated. ---")

In [ ]:
# Cell 5: Feature Extraction (MFCCs)
import librosa
import numpy as np
# os is already imported from previous cells, but adding for clarity of this cell's dependencies

print("\n--- Step 5: Feature Extraction (MFCCs) ---")

# These will be set by the debugging snippet if successful
mfcc_dim1 = None
mfcc_dim2 = None

def extract_mfccs(file_path, n_mfcc=40, target_duration=3, sr=22050):
    """
    Extracts MFCCs from an audio file, resampling and padding/truncating
    to ensure consistent output shape.

    Args:
        file_path (str): Path to the audio file.
        n_mfcc (int): Number of MFCCs to extract.
        target_duration (int): Target duration in seconds for padding/truncation.
        sr (int): Target sampling rate for resampling.

    Returns:
        np.array: Padded/truncated MFCCs, or None if an error occurs.
    """
    try:
        # Load audio with original sampling rate first, then resample
        y, original_sr = librosa.load(file_path, sr=None)

        # Resample to the target sampling rate for consistency
        if original_sr != sr:
            y = librosa.resample(y=y, orig_sr=original_sr, target_sr=sr)

        # Calculate target number of samples for padding/truncation
        target_length_samples = int(sr * target_duration)

        # Pad or truncate audio to the target length
        if len(y) > target_length_samples:
            y = y[:target_length_samples] # Truncate if longer
        else:
            y = np.pad(y, (0, max(0, target_length_samples - len(y))), "constant") # Pad if shorter

        # Extract MFCCs
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

        # Fixed target number of frames for MFCCs
        # For RAVDESS, 3 seconds of audio at 22050Hz, with default hop_length=512
        # typically results in around 130 frames (ceil(3 * 22050 / 512) = 130)
        fixed_mfcc_frames = 130

        if mfccs.shape[1] > fixed_mfcc_frames:
            mfccs = mfccs[:, :fixed_mfcc_frames] # Truncate if more frames than target
        else:
            pad_width = fixed_mfcc_frames - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant') # Pad if fewer frames

        return mfccs

    except FileNotFoundError:
        # This error should ideally be caught earlier if paths in DataFrame are wrong
        print(f"ERROR: File not found during MFCC extraction: {file_path}")
        return None
    except Exception as e:
        # Catch any other librosa or numpy processing errors
        print(f"ERROR: Failed to process audio file '{file_path}': {e}")
        return None

# --- IMPORTANT DEBUGGING SNIPPET FOR INITIAL MFCC EXTRACTION ---
print("\n--- Debugging MFCC Extraction for a single file ---")
if not df.empty:
    first_file_path = df['file_path'].iloc[0]
    print(f"Attempting to process first file: {first_file_path}")
    try:
        test_mfccs = extract_mfccs(first_file_path)
        if test_mfccs is not None:
            print(f"SUCCESS: MFCCs extracted for first file. Shape: {test_mfccs.shape}")
            # Set global dimensions based on successful test
            mfcc_dim1, mfcc_dim2 = test_mfccs.shape
            print(f"Determined MFCC dimensions: n_mfcc={mfcc_dim1}, time_frames={mfcc_dim2}")
        else:
            print(f"FATAL ERROR: Could not extract MFCCs for first file. Check error messages above.")
            print("This indicates a problem with the audio file or the 'extract_mfccs' function itself.")
            import sys
            sys.exit("Exiting due to critical MFCC extraction failure on first file.")
    except Exception as e:
        print(f"FATAL ERROR: An unexpected error occurred during direct test of MFCC extraction for the first file: {e}")
        import sys
        sys.exit("Exiting due to unexpected error during first file MFCC test.")
else:
    print("FATAL ERROR: DataFrame is empty. No files to process for MFCC extraction. Please ensure Cell 4 ran successfully.")
    import sys
    sys.exit("Exiting due to empty DataFrame for MFCC extraction.")
print("--- End Debugging Snippet ---\n")


print("Starting full MFCC extraction for all files. This may take a few minutes...")
df['mfccs'] = df['file_path'].apply(extract_mfccs)

# Remove rows where MFCC extraction failed (returns None)
initial_audio_count = len(df)
df.dropna(subset=['mfccs'], inplace=True)
final_audio_count = len(df)

print(f"\nTotal audio files initially in DataFrame: {initial_audio_count}")
print(f"Total audio files successfully processed for MFCCs: {final_audio_count}")
if initial_audio_count > final_audio_count:
    print(f"WARNING: {initial_audio_count - final_audio_count} files failed MFCC extraction and were removed.")

# Final check for data availability after full processing
if not df.empty:
    # Ensure mfcc_dim1 and mfcc_dim2 are correctly set for subsequent cells
    # They should already be set by the debug snippet, but ensure consistency
    if not (mfcc_dim1, mfcc_dim2) == df['mfccs'].iloc[0].shape:
        print("WARNING: MFCC dimensions changed after full processing. Resetting.")
        mfcc_dim1, mfcc_dim2 = df['mfccs'].iloc[0].shape
    print(f"MFCC dimensions for model input confirmed: n_mfcc={mfcc_dim1}, time_frames={mfcc_dim2}")
    print("\n--- Step 5 Complete: MFCCs extracted. ---")
else:
    print("CRITICAL ERROR: No valid MFCCs extracted after full processing. DataFrame is empty.")
    print("This implies a widespread issue with your audio files or the extraction function.")
    import sys
    sys.exit("Exiting due to no valid MFCCs after full processing.")

In [ ]:
# Cell 6: Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np # Ensure numpy is imported

print("\n--- Step 6: Data Preprocessing ---")

# Convert MFCCs list of arrays into a single NumPy array
# X needs to be (samples, n_mfcc, time_frames) for Conv1D, or (samples, time_frames, n_mfcc)
# Our current MFCCs are (n_mfcc, time_frames). So, X will be (samples, n_mfcc, time_frames)
X = np.array(df['mfccs'].tolist())

# Conv1D expects input shape (batch, timesteps, features) or (batch, features, timesteps)
# If we treat n_mfcc as features and time_frames as timesteps: X.shape becomes (samples, time_frames, n_mfcc)
# So, we should transpose the MFCCs to (time_frames, n_mfcc) before creating the array, or transpose X later.
# Let's reshape X to (samples, time_frames, n_mfcc) as this is more common for Conv1D.
X = X.transpose(0, 2, 1) # Transpose from (samples, n_mfcc, time_frames) to (samples, time_frames, n_mfcc)

# Now add the channel dimension for Conv1D, usually 1 for mono audio features
X = np.expand_dims(X, -1) # Shape: (samples, time_frames, n_mfcc, 1)

# Ensure mfcc_dim1 and mfcc_dim2 are updated to reflect the time_frames and n_mfcc after transpose
# For Conv1D input_shape will be (time_frames, n_mfcc, 1)
mfcc_timesteps = X.shape[1] # This is now the time_frames
mfcc_features = X.shape[2]  # This is now the n_mfcc
input_shape_for_conv1d = (mfcc_timesteps, mfcc_features)

print(f"Adjusted MFCC dimensions for Conv1D input_shape: (timesteps, features) = {input_shape_for_conv1d}")

# Initialize LabelEncoder to convert emotion names (strings) to numerical labels
le = LabelEncoder()
# Fit LabelEncoder on all unique emotion names and transform them
y_encoded = le.fit_transform(df['emotion'])
# Convert numerical labels to one-hot encoded format
y = to_categorical(y_encoded)

num_classes = y.shape[1]
print(f"Number of emotion classes: {num_classes}")
print(f"Emotion labels (original order): {le.classes_}")

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y_encoded)

print(f"\nShape of training features (X_train): {X_train.shape}")
print(f"Shape of validation features (X_val): {X_val.shape}")
print(f"Shape of training labels (y_train): {y_train.shape}")
print(f"Shape of validation labels (y_val): {y_val.shape}")
print("\n--- Step 6 Complete: Data preprocessed. ---")

In [ ]:
# Cell 7: Model Definition (CNN)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam # Explicitly import Adam

print("\n--- Step 7: Model Definition (CNN) ---")

# Ensure mfcc_timesteps and mfcc_features are set from previous step (Cell 6)
# These variables come from the shape of X_train after preprocessing in Cell 6.
# If you run this cell independently, ensure Cell 6 has been run first.
if 'mfcc_timesteps' not in globals() or 'mfcc_features' not in globals():
    print("FATAL ERROR: MFCC dimensions (mfcc_timesteps, mfcc_features) not found. Please run Cell 6 first.")
    import sys
    sys.exit("Exiting.")

model = Sequential([
    # Input shape for Conv1D is (timesteps, features)
    # X_train is (samples, time_frames, n_mfcc, 1) -> input_shape for Conv1D will be (time_frames, n_mfcc)
    Conv1D(filters=128, kernel_size=7, activation='relu', input_shape=(mfcc_timesteps, mfcc_features)),
    BatchNormalization(), # Added Batch Normalization after Conv1D
    MaxPooling1D(pool_size=2),
    Dropout(0.3), # Adjusted dropout

    Conv1D(filters=256, kernel_size=5, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Conv1D(filters=512, kernel_size=3, activation='relu'), # Smaller kernel for deeper layer
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Flatten(),
    Dense(512, activation='relu'), # Increased Dense layer size
    BatchNormalization(), # Added Batch Normalization for Dense layer
    Dropout(0.6), # Increased dropout for dense layer to combat potential overfitting
    Dense(num_classes, activation='softmax') # Output layer
])

model.compile(optimizer=Adam(learning_rate=0.0005), # Slightly lower initial learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks for better training
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True) # Increased patience
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=12, min_lr=0.000005) # Increased patience

print("Model Architecture Summary:")
model.summary()
print("\n--- Step 7 Complete: Model defined. ---")

In [ ]:
# Cell 8: Model Training
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
import numpy as np # Ensure numpy is imported

print("\n--- Step 8: Model Training ---")

# Compute class weights
# y_encoded_train contains the integer labels for the training set
# You need the integer labels, not one-hot encoded, for compute_class_weight
# If y_train is one-hot, convert it back to integer labels first
y_encoded_train_labels = np.argmax(y_train, axis=1)

class_weights = class_weight.compute_class_weight(
    class_weight='balanced', # 'balanced' automatically adjusts weights inversely proportional to class frequencies
    classes=np.unique(y_encoded_train_labels),
    y=y_encoded_train_labels
)
class_weights_dict = dict(enumerate(class_weights)) # Map integer label to its weight

print(f"Computed class weights: {class_weights_dict}")
# You might manually inspect these. If neutral is heavily underrepresented, its weight will be higher.
# Even if balanced, 'balanced' weights can still help hard-to-classify classes.


history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,
                    batch_size=32,
                    callbacks=[early_stopping, reduce_lr],
                    class_weight=class_weights_dict, # ADD THIS LINE
                    verbose=1)

print("\n--- Step 8 Complete: Training finished. ---")

# Plot training history (rest of the code remains the same)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Cell 9: Model Evaluation
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score
import matplotlib.pyplot as plt # Ensure plt is imported

print("\n--- Step 9: Model Evaluation ---")

# Evaluate on validation data
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Overall Accuracy: {val_accuracy:.4f}")

# Predictions for confusion matrix and F1 score
y_pred_probs = model.predict(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_val, axis=1)

# Convert numerical labels back to emotion names for clarity
y_pred_labels = le.inverse_transform(y_pred)
y_true_labels = le.inverse_transform(y_true)

# --- Confusion Matrix ---
print("\n--- Confusion Matrix ---")
cm = confusion_matrix(y_true_labels, y_pred_labels, labels=le.classes_)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# --- Classification Report ---
print("\n--- Classification Report ---")
report = classification_report(y_true_labels, y_pred_labels, target_names=le.classes_, output_dict=True)
print(classification_report(y_true_labels, y_pred_labels, target_names=le.classes_))

# --- Calculate F1 Score (weighted average for multi-class) ---
f1_weighted = f1_score(y_true_labels, y_pred_labels, average='weighted')
print(f"\nWeighted F1 Score: {f1_weighted:.4f}")

# --- Check individual class accuracies ---
print("\n--- Individual Class Accuracies ---")
class_accuracies = {}
for class_name in le.classes_:
    true_for_class = y_true_labels == class_name
    predicted_correctly_for_class = (y_pred_labels == class_name) & true_for_class
    total_samples_in_class = np.sum(true_for_class)

    if total_samples_in_class > 0:
        class_accuracy = np.sum(predicted_correctly_for_class) / total_samples_in_class
    else:
        class_accuracy = 0 # Handle cases where a class might have no samples
    class_accuracies[class_name] = class_accuracy
    print(f"Accuracy for class '{class_name}': {class_accuracy:.4f}")

print("\n--- Project Evaluation Criteria Check ---")
print(f"Overall Validation Accuracy: {val_accuracy * 100:.2f}% (Target: > 80%)")
print(f"Weighted F1 Score: {f1_weighted * 100:.2f}% (Target: > 80%)")

all_class_accuracy_met = True
for class_name, acc in class_accuracies.items():
    if acc < 0.75:
        all_class_accuracy_met = False
        print(f"FAIL: Class '{class_name}' accuracy ({acc * 100:.2f}%) is below 75%.")
    else:
        print(f"PASS: Class '{class_name}' accuracy ({acc * 100:.2f}%) is above 75%.")

if val_accuracy > 0.80 and f1_weighted > 0.80 and all_class_accuracy_met:
    print("\nCONGRATULATIONS! All project criteria on validation set are potentially met.")
else:
    print("\nKEEP WORKING! Not all project criteria on validation set are met yet.")
    print("Consider further tuning, data augmentation, or a more complex model.")

print("\n--- Step 9 Complete: Evaluation finished. ---")

In [ ]:
# Cell 10: Plot Accuracy (Final Plot)
import matplotlib.pyplot as plt

# Plot accuracy
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
model.save('vk.h5')

In [ ]:
# Cell 11: Save the Trained Model and LabelEncoder

import os
import joblib # A robust library for saving and loading Python objects

print("\n--- Step 11: Saving Model and LabelEncoder ---")

# Define paths to save your model and label encoder in your Google Drive
# It's good practice to create a specific folder for deployment assets
deployment_assets_dir = "/content/drive/MyDrive/emotion_audio_deployment"
os.makedirs(deployment_assets_dir, exist_ok=True)

model_save_path = os.path.join(deployment_assets_dir, "ravdess_emotion_model.keras")
label_encoder_save_path = os.path.join(deployment_assets_dir, "label_encoder.pkl")

try:
    # 1. Save the Keras model
    model.save(model_save_path)
    print(f"Model successfully saved to: {model_save_path}")
except Exception as e:
    print(f"ERROR: Could not save the model: {e}")

try:
    # 2. Save the LabelEncoder
    # The 'le' object was created in Cell 6
    joblib.dump(le, label_encoder_save_path)
    print(f"LabelEncoder successfully saved to: {label_encoder_save_path}")
except NameError:
    print("ERROR: 'le' (LabelEncoder) variable not found. Please ensure Cell 6 has been run.")
except Exception as e:
    print(f"ERROR: Could not save the LabelEncoder: {e}")

print("\n--- Step 11 Complete: Assets saved for deployment. ---")

In [ ]:
%%writefile app.py
import streamlit as st
# ... rest of your app.py code ...

In [ ]:
%%writefile app.py
import streamlit as st# app.py

import streamlit as st
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib
import os
import io # To handle file-like objects from Streamlit uploader

# --- 1. Load the Model and LabelEncoder ---
# These paths are relative to where your app.py will be run or deployed.
# Ensure your 'ravdess_emotion_model.keras' and 'label_encoder.pkl' are
# in the same directory as your app.py file when deploying.
MODEL_PATH = 'ravdess_emotion_model.keras'
LABEL_ENCODER_PATH = 'label_encoder.pkl'

# Check if model and encoder files exist (important for local testing and deployment)
if not os.path.exists(MODEL_PATH):
    st.error(f"Error: Model file '{MODEL_PATH}' not found. Please ensure it's in the same directory.")
    st.stop() # Stop the app if crucial files are missing
if not os.path.exists(LABEL_ENCODER_PATH):
    st.error(f"Error: LabelEncoder file '{LABEL_ENCODER_PATH}' not found. Please ensure it's in the same directory.")
    st.stop()

@st.cache_resource # Cache the model loading for performance
def load_my_model(path):
    return load_model(path)

@st.cache_resource # Cache the label encoder loading
def load_my_label_encoder(path):
    return joblib.load(path)

model = load_my_model(MODEL_PATH)
le = load_my_label_encoder(LABEL_ENCODER_PATH)

# --- 2. Define Preprocessing Function (MUST MATCH Colab Cell 5 & 6) ---
def preprocess_audio(audio_file_path_or_buffer, n_mfcc=40, target_duration=3, sr=22050):
    """
    Extracts MFCCs from an audio file (or buffer), resampling and padding/truncating
    to ensure consistent output shape, ready for model prediction.
    This function MUST EXACTLY MATCH the logic in your Colab notebook's extract_mfccs
    and subsequent reshaping.
    """
    try:
        # librosa.load can handle file paths or file-like objects (like BytesIO from st.file_uploader)
        y, original_sr = librosa.load(audio_file_path_or_buffer, sr=None)

        # Resample if necessary
        if original_sr != sr:
            y = librosa.resample(y=y, orig_sr=original_sr, target_sr=sr)

        # Calculate target number of samples for padding/truncation
        target_length_samples = int(sr * target_duration)

        # Pad or truncate audio to the target length
        if len(y) > target_length_samples:
            y = y[:target_length_samples]
        else:
            y = np.pad(y, (0, max(0, target_length_samples - len(y))), "constant")

        # Extract MFCCs
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

        # Fixed target number of frames for MFCCs (from Colab Cell 5)
        fixed_mfcc_frames = 130

        if mfccs.shape[1] > fixed_mfcc_frames:
            mfccs = mfccs[:, :fixed_mfcc_frames]
        else:
            pad_width = fixed_mfcc_frames - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

        # Reshape for Conv1D input (from Colab Cell 6)
        # From (n_mfcc, time_frames) to (1, time_frames, n_mfcc, 1)
        mfccs = mfccs.transpose(1, 0) # (time_frames, n_mfcc)
        mfccs = np.expand_dims(mfccs, axis=0) # (1, time_frames, n_mfcc) - for batch
        mfccs = np.expand_dims(mfccs, axis=-1) # (1, time_frames, n_mfcc, 1) - add channel dim

        return mfccs

    except Exception as e:
        st.error(f"Error processing audio: {e}")
        return None

# --- 3. Streamlit UI ---
st.title("Emotion Recognition from Speech/Song")
st.markdown("Upload an audio file (WAV) and let the CNN model predict its emotion!")

uploaded_file = st.file_uploader("Choose an audio file...", type=["wav"])

if uploaded_file is not None:
    # Display uploaded audio
    st.audio(uploaded_file, format='audio/wav')

    # Create a BytesIO object from the uploaded file
    audio_bytes = io.BytesIO(uploaded_file.read())

    # Preprocess the audio
    features = preprocess_audio(audio_bytes)

    if features is not None:
        st.write("Audio processing complete. Predicting emotion...")
        # Make prediction
        prediction = model.predict(features)
        predicted_class_index = np.argmax(prediction, axis=1)[0]
        predicted_emotion = le.inverse_transform([predicted_class_index])[0]
        confidence = np.max(prediction) * 100

        st.subheader("Prediction Result:")
        st.success(f"**Predicted Emotion:** {predicted_emotion.upper()}")
        st.info(f"**Confidence:** {confidence:.2f}%")

        # Optional: Display all probabilities
        st.markdown("---")
        st.subheader("All Emotion Probabilities:")
        prob_df = pd.DataFrame({
            'Emotion': le.classes_,
            'Probability': prediction[0] * 100
        }).sort_values(by='Probability', ascending=False)
        st.dataframe(prob_df.style.format({"Probability": "{:.2f}%"}), hide_index=True)


    else:
        st.error("Could not extract features from the audio file. Please try another file.")

st.markdown("---")
st.markdown("Note: This model was trained on the RAVDESS dataset and performs best on similar audio.")